## Keras Embedding Layer Example

In [ ]:
sentences = ['멋있어 최고야 짱이야 감탄이다', '헛소리 지껄이네', '닥쳐 자식아', \
             '우와 대단하다', '우수한 성적', '형편없다', '최상의 퀄리티']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [ ]:
from keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1

print(vocab_size)

In [ ]:
X_encoded = t.texts_to_sequences(sentences)
print(X_encoded)

In [ ]:
max_len=max(len(l) for l in X_encoded)
print(max_len)

In [ ]:
from keras.preprocessing.sequence import pad_sequences
X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
print(X_train)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten

model = Sequential()
model.add(Embedding(vocab_size, 4, input_length=max_len)) # 모든 임베딩 벡터는 4차원을 가지게됨.
model.add(Flatten()) # Dense의 입력으로 넣기위함임.
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

## Pre-Trained GloVe Embedding 사용하기

In [ ]:
sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', \
             'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

from keras.preprocessing.text import Tokenizer
t = Tokenizer()
t.fit_on_texts(sentences)
vocab_size = len(t.word_index) + 1

X_encoded = t.texts_to_sequences(sentences)
max_len=max(len(l) for l in X_encoded)

from keras.preprocessing.sequence import pad_sequences
X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
print(X_train)

In [ ]:
n = 0
f = open('./glove.6B.100d.txt', encoding="utf8")
for line in f:
    word_vector = line.split() # 각 줄을 읽어와서 word_vector에 저장.
    print(word_vector) # 각 줄을 출력
    word = word_vector[0] # word_vector에서 첫번째 값만 저장
    print(word) # word_vector의 첫번째 값만 출력
    n=n+1
    if n==2:
        break
f.close()

In [ ]:
print(type(word_vector))
print(len(word_vector))

In [ ]:
import numpy as np
embedding_dict = dict()
f = open('./glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') 
    # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()
print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

In [ ]:
print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))
# 단어 집합 크기의 행과 100개의 열을 가지는 행렬 생성. 값은 전부 0으로 채워진다.
np.shape(embedding_matrix)

In [ ]:
print(t.word_index.items())

In [ ]:
for word, i in t.word_index.items(): 
    # 훈련 데이터의 단어 집합에서 단어를 1개씩 꺼내온다.
    temp = embedding_dict.get(word) 
    # 단어(key) 해당되는 임베딩 벡터의 100개의 값(value)를 임시 변수에 저장
    if temp is not None:
        embedding_matrix[i] = temp 
        # 임수 변수의 값을 단어와 맵핑되는 인덱스의 행에 삽입

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten

model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False)

In [ ]:
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)